<a href="https://colab.research.google.com/github/stasiek-j/SAD2/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import distributions
import sklearn, torch
torch.manual_seed(42)

In [ ]:
K = 3
N = 2
size = (K, N)

In [ ]:
component = distributions.Independent(distributions.Normal(
    torch.randn(size), torch.rand(size)), 1)
mixture = distributions.Categorical(torch.ones(K,))

gmm = distributions.MixtureSameFamily(mixture, component)

In [ ]:
gmm.sample()

tensor([0.8017, 0.5883])

In [ ]:
d = gmm.sample((10,))
print(d)
d.shape

tensor([[-0.0876,  0.8052],
        [-2.1717,  0.6077],
        [-0.5629,  0.4603],
        [ 0.3422,  0.3295],
        [ 0.8366, -0.2355],
        [-0.1124, -0.1567],
        [ 0.9697, -0.2746],
        [ 1.4971,  0.4033],
        [ 0.9680,  0.1806],
        [ 0.4519,  0.5627]])


torch.Size([10, 2])

In [ ]:
parameters = [1, [.5], [.5]]

In [ ]:
def sample_multivariate():
  pass

In [ ]:
def e_step(data, params):
  for i in range(data.shape[0]):
    x = data[i, 0]
    y = data[i, 1]
    pcluster1 = 
    print(x)

  pass


def m_step():
  pass

In [ ]:
e_step(d, parameters)

tensor(-0.3813)
tensor(0.8443)
tensor(-1.8538)
tensor(3.2614)
tensor(0.1813)
tensor(1.6670)
tensor(3.2550)
tensor(0.8692)
tensor(-0.8103)
tensor(2.5540)
